In [155]:
from aocd import submit
import numpy as np
import re
import sys

In [325]:
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)

5000


In [2]:
YYYY=2021
DD=23

In [5]:
def load(filename):
    with open(filename, "r") as f:
        lines = [l.rstrip("\n") for l in f]
    return lines
test1 = load("test1.dat")
test1

['#############',
 '#...........#',
 '###B#C#B#D###',
 '  #A#D#C#A#',
 '  #########']

In [259]:
data_in = load("input.dat")
data_in2 = load("input2.dat")

In [260]:
data_in2

['#############',
 '#...........#',
 '###B#C#A#D###',
 '  #D#C#B#A#',
 '  #D#B#A#C#',
 '  #B#C#D#A#',
 '  #########']

# Teil a

In [391]:
def read_conf(config):
    free=[]
    x = [(1, 3),(1, 5),(1, 7),(1, 9)] # RULE1
    a,b,c,d=[],[],[],[]
    for i, row in enumerate(config):
        for j,col in enumerate(row):
            if col == ".":
                free.append((i,j))
            elif col == "A":
                a.append((i,j))
            elif col == "B":
                b.append((i,j))
            elif col == "C":
                c.append((i,j))
            elif col == "D":
                d.append((i,j))
    conf ={"flur": free[:], "free": free, "A":a,"B":b,"C":c,"D":d, "X":x}
    
    conf["HA"] = [(2, 3), (3, 3)]
    conf["HB"] = [(2, 5), (3, 5)]
    conf["HC"] = [(2, 7), (3, 7)]
    conf["HD"] = [(2, 9), (3, 9)]

    return conf

def clean(c):
    for i, x in enumerate(["A","B","C","D"]):
        if (3,3+(i*2)) in c[x] and len(c[x]) == 2:
            c[x].remove((3,3+(i*2)))
        if (2,3+(i*2)) in c[x] and len(c[x]) == 1:
            c[x].remove((2,3+(i*2)))

def clean2(c):
    if (3,3) in c["A"]:
        c["A"].remove((3,3))
    if (2,3) in c["A"] and len(c["A"]) == 1:
        c["A"].remove((2,3))
    if (3,5) in c["B"]:
        c["B"].remove((3,5))
    if (2,5) in c["B"] and len(c["B"]) == 1:
        c["B"].remove((2,5))
    if (3,7) in c["C"]:
        c["C"].remove((3,7))
    if (2,7) in c["C"] and len(c["C"]) == 1:
        c["C"].remove((2,7))
    if (3,9) in c["D"]:
        c["D"].remove((3,9))
    if (2,9) in c["D"] and len(c["D"]) == 1:
        c["D"].remove((2,9))

def finish(c):
    for x in ["A", "B", "C", "D"]:
        if len(c[x]) > 0:
            return False
    return True

def get_neighbour_space(pos, config, visited, steps):
    x,y=pos
    for n in [(x-1, y),(x+1,y),(x,y-1),(x,y+1)]:
        if (n in visited and visited[n] <=steps+1) or n not in config["free"]:
            continue
        visited[n]=steps+1
        get_neighbour_space(n, config, visited, steps+1)

def move(c, pos1, pos2, steps):
    for i, x in enumerate(["A", "B", "C", "D"]):
        cps = 10**i #costs per step
        if pos1 in c[x] and pos2 not in c["X"]:
            c[x].remove(pos1)
            c["free"].append(pos1)
            c[x].append(pos2)
            c["free"].remove(pos2)
            return steps*cps
    return 10000

def get_amphis(c):
    l = []
    l += c["A"]
    l += c["B"]
    l += c["C"]
    l += c["D"]
    return l

def get_type(a,c):
    for x in ["A", "B", "C", "D"]:
        if a in c[x]:
            return x
    return "Free"

def homebase(t, pos, c):
    if pos in c["flur"]:
        return False
    return pos in c["H"+t] and sideroom_free(t, c) and check_hb(t,pos,c)

def sideroom_free(t, c):
    for p in c["H"+t]:
        tt = get_type(p,c)
        if tt != "Free" and tt != t:
            return False
    return True
    
    
def get_moves(c):
    moves = {}
    amphis = get_amphis(c)
    for a in amphis:
        vorschlag = {}
        get_neighbour_space(a, c, vorschlag, 0)
        m = {}
        t = get_type(a, c)
        #p = get_lowest(t, c)
        for pos2, s in vorschlag.items():
            if homebase(t, pos2, c): # immediately jump home if possible
         #       if pos2 == p:
                return {a:{pos2:s}}
                
            if a in c["flur"] and pos2 in c["flur"]:
                continue # no moves on the floor
            #if not homebase(a, pos2,c):
            #    continue # dont go in wrong homebse
            if a in c["flur"] and not homebase(t, pos2, c):
                continue
            if (a not in c["flur"] and not homebase(t, pos2, c)) and pos2 not in c["flur"]:
                continue
            if pos2 in c["X"]:
                continue
            m[pos2] = s
        moves[a] = m
    return moves

def get_lowest(t, c):
    for p in c["H"+t][::-1]:
        tt = get_type(p,c)
        if tt == "Free":
            return p
    return (0,0)

def check_hb(t,pos,c):
    i = "ABCD".find(t)
    if (3,3+(i*2)) == pos and len(c[t]) == 2:
        return True
    if (2,3+(i*2)) == pos and len(c[t]) == 1:
        return True
    return False


def play(config, costs, s=0):
    if s > 50:
        return 33000
    clean(config)
    if costs > 33000:
        return costs
    if finish(config):
        #print("OK")
        return costs
    best = 33000
    clean(config)
    moves = get_moves(config)
    if len(moves) == 0:
        #print("ERR")
        return best
    for pos1, dest in get_moves(config).items():
        for pos2, steps in dest.items():
            c = {key: value[:] for key, value in config.items()}
            m_cost = move(c, pos1,pos2, steps)
            #print(f"move {pos1, pos2}")
            #print(c)
            new_costs = play(c, costs + m_cost, s+1)
            #print(f"{new_costs} {costs} + {m_cost}")
            if new_costs < best:
                best = new_costs
    #print(best)
    return best

In [392]:
def solution_a(data):
    conf = read_conf(data)
    return play(conf,0)

In [393]:
conf = read_conf(test1)
move(conf, (2,7), (1,4), 4)
clean(conf)
get_moves(conf)
#conf

{(2, 5): {(2, 7): 4}}

In [394]:
solution_a(test1)

12521

In [ ]:
solution_a(data_in)

In [252]:
submit(solution_a(data_in),part="a",day=DD, year=YYYY)

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


<Response [200]>

# Teil b

In [399]:
def read_conf(config):
    free=[]
    x = [(1, 3),(1, 5),(1, 7),(1, 9)] # RULE1
    a,b,c,d=[],[],[],[]
    for i, row in enumerate(config):
        for j,col in enumerate(row):
            if col == ".":
                free.append((i,j))
            elif col == "A":
                a.append((i,j))
            elif col == "B":
                b.append((i,j))
            elif col == "C":
                c.append((i,j))
            elif col == "D":
                d.append((i,j))
    conf ={"flur": free[:], "free": free, "A":a,"B":b,"C":c,"D":d, "X":x}
    
    conf["HA"] = [(2, 3), (3, 3), (4, 3), (5, 3)]
    conf["HB"] = [(2, 5), (3, 5), (4, 5), (5, 5)]
    conf["HC"] = [(2, 7), (3, 7), (4, 7), (5, 7)]
    conf["HD"] = [(2, 9), (3, 9), (4, 9), (5, 9)]

    return conf

def clean(c):
    for i, x in enumerate(["A","B","C","D"]):
        c[x].sort()
        if (5,3+(i*2)) in c[x]:
            c[x].remove((5,3+(i*2)))
        if (4,3+(i*2)) in c[x] and len(c[x]) == 3:
            c[x].remove((4,3+(i*2)))
        if (3,3+(i*2)) in c[x] and len(c[x]) == 2:
            c[x].remove((3,3+(i*2)))
        if (2,3+(i*2)) in c[x] and len(c[x]) == 1:
            c[x].remove((2,3+(i*2)))

def check_hb(t, pos, c):
    i = "ABCD".find(t)
    if (5,3+(i*2)) == pos and len(c[t]) == 4:
        return True
    if (4,3+(i*2)) == pos and len(c[t]) == 3:
        return True
    if (3,3+(i*2)) == pos and len(c[t]) == 2:
        return True
    if (2,3+(i*2)) == pos and len(c[t]) == 1:
        return True
    return False

def finish(c):
    for x in ["A", "B", "C", "D"]:
        if len(c[x]) > 0:
            return False
    return True

def get_neighbour_space(pos, config, visited, steps):
    x,y=pos
    for n in [(x-1, y),(x+1,y),(x,y-1),(x,y+1)]:
        if (n in visited and visited[n] <=steps+1) or n not in config["free"]:
            continue
        visited[n]=steps+1
        get_neighbour_space(n, config, visited, steps+1)

def move(c, pos1, pos2, steps):
    for i, x in enumerate(["A", "B", "C", "D"]):
        cps = 10**i #costs per step
        if pos1 in c[x] and pos2 not in c["X"]:
            c[x].remove(pos1)
            c["free"].append(pos1)
            c[x].append(pos2)
            c["free"].remove(pos2)
            return steps*cps
    return 10000

def get_amphis(c):
    l = []
    l += c["A"]
    l += c["B"]
    l += c["C"]
    l += c["D"]
    return l

def get_type(a,c):
    for x in ["A", "B", "C", "D"]:
        if a in c[x]:
            return x
    return "Free"

def homebase(t, pos, c):
    if pos in c["flur"]:
        return False
    return pos in c["H"+t] and sideroom_free(t, c) and check_hb(t,pos,c)

def sideroom_free(t, c):
    for p in c["H"+t]:
        tt = get_type(p,c)
        if tt != "Free" and tt != t:
            return False
    return True

def get_lowest(t, c):
    for p in c["H"+t][::-1]:
        tt = get_type(p,c)
        if tt == "Free":
            return p
    return (0,0)

    
    
def get_moves(c):
    moves = {}
    amphis = get_amphis(c)
    for a in amphis:
        vorschlag = {}
        get_neighbour_space(a, c, vorschlag, 0)
        #vorschlag = {k: v for k, v in sorted(vorschlag.items(), key=lambda item: item[1])}
        m = {}
        t = get_type(a, c)
        #p = get_lowest(t, c)
        for pos2, s in vorschlag.items():
            if homebase(t, pos2, c): # immediately jump home if possible
                #if pos2 == p:
                return {a:{pos2:s}}
            if a in c["flur"] and pos2 in c["flur"]:
                continue # no moves on the floor
            if a in c["flur"] and not homebase(t, pos2, c):
                continue
            if (a not in c["flur"] and not homebase(t, pos2, c)) and pos2 not in c["flur"]:
                continue
            if pos2 in c["X"]:
                continue
            m[pos2] = s
        moves[a] = m
    return moves

def play(config, costs, s=1):
    if s > 100:
        return 80000
    clean(config)
    if costs > 80000:
        return costs
    if finish(config):
        print(f"OK  {costs}")
        return costs
    best = 80000
    moves = get_moves(config)
    if len(moves) == 0:
        #print("ERR")
        return best
    for pos1, dest in get_moves(config).items():
        for pos2, steps in dest.items():
            c = {key: value[:] for key, value in config.items()}
            m_cost = move(c, pos1,pos2, steps)
            new_costs = play(c, costs + m_cost, s+1)
            if new_costs < best:
                best = new_costs
    return best

In [400]:
def solution_b(data):
    conf = read_conf(data)
    print(data)
    return play(conf,0)

In [401]:
c=read_conf(data_in2)
c

{'flur': [(1, 1),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (1, 6),
  (1, 7),
  (1, 8),
  (1, 9),
  (1, 10),
  (1, 11)],
 'free': [(1, 1),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (1, 6),
  (1, 7),
  (1, 8),
  (1, 9),
  (1, 10),
  (1, 11)],
 'A': [(2, 7), (3, 9), (4, 7), (5, 9)],
 'B': [(2, 3), (3, 7), (4, 5), (5, 3)],
 'C': [(2, 5), (3, 5), (4, 9), (5, 5)],
 'D': [(2, 9), (3, 3), (4, 3), (5, 7)],
 'X': [(1, 3), (1, 5), (1, 7), (1, 9)],
 'HA': [(2, 3), (3, 3), (4, 3), (5, 3)],
 'HB': [(2, 5), (3, 5), (4, 5), (5, 5)],
 'HC': [(2, 7), (3, 7), (4, 7), (5, 7)],
 'HD': [(2, 9), (3, 9), (4, 9), (5, 9)]}

In [ ]:
solution_b(data_in2)

['#############', '#...........#', '###B#C#A#D###', '  #D#C#B#A#', '  #D#B#A#C#', '  #B#C#D#A#', '  #########']
OK  50752
OK  49552
OK  49352
OK  49552
OK  50792
OK  49432
OK  49232
OK  49432
OK  49552
OK  49352
OK  49452
OK  49492
OK  50052
OK  50092
OK  49852
OK  49892
OK  49252
OK  49292
OK  49552
OK  49352
OK  49552
OK  49452
OK  49492
OK  50052
OK  50092
OK  49852
OK  49892
OK  50052
OK  50092
OK  50752
OK  50792
OK  50232
OK  50032
OK  50232
OK  50752
OK  50792
OK  50752
OK  50732
OK  50772
OK  50792
OK  50772
OK  56746
OK  50752
OK  56746
OK  56746
OK  56766
OK  50772
OK  56766
OK  56766
OK  56806
OK  50812
OK  56806
OK  56806
OK  49552
OK  49352


In [ ]:
submit(solution_b(data_in2),part="b",day=DD, year=YYYY)